In [1]:
import pandas as pd
import openpyxl
from datetime import datetime

Feature 1: read in 2 datasets. I pulled the Annual visitation numbers for Mammoth Cave as my first dataset. This can be found at irma.nps.gov - full link included in ReadMe.  

In [2]:
mammoth_df = pd.read_excel('park_visitation_1904_current.xlsx')
mammoth_df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Mammoth Cave NP,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,Year,Recreation Visits
3,NaN,NaN,1936,57775
4,NaN,NaN,1937,75434


Okay the formatting is weird here, but I'll fix that later. Now to import the second dataset, gas prices. This is an excel file with two separate sheets and the data I want is on sheet 2, titled "Data 1". 

Feature 1, dataset 2: Annual gas prices.

In [3]:
gas_df = pd.read_excel('gas_data_1994_current.xls', 'Data 1')
gas_df.head()

,Back to Contents,Data 1: U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)
0,Sourcekey,EMM_EPM0_PTE_NUS_DPG
1,Date,U.S. All Grades All Formulations Retail Gasoli...
2,1993-06-30 00:00:00,NaN
3,1994-06-30 00:00:00,1.078
4,1995-06-30 00:00:00,1.158


This dataset only goes back to 1994, so I guess I need to exclude annual visitor info before that date. Gotta name the columns and then filter by year.

Feature 2: data cleaning. 

In [4]:
mammoth_df_1994 = mammoth_df.rename(columns={'Unnamed: 0': 'Placeholder', 'Unnamed: 1': 'NP', 'Unnamed: 2': 'year', 'Unnamed: 3': 'recreation_visits'})
mammoth_df_1994.columns.tolist()



['Placeholder', 'NP', 'year', 'recreation_visits']

The first two columns are kinda pointless. Let's drop those. 

In [5]:
mammoth_df_1994 = mammoth_df_1994.drop(columns=['Placeholder', 'NP'])

In [6]:
mammoth_df_1994.drop([0,1,2,90], axis=0, inplace=True)
mammoth_df_1994.head()

,year,recreation_visits
3,1936,57775
4,1937,75434
5,1938,120692
6,1939,116516
7,1940,117751


In [7]:
print(mammoth_df_1994)

    year recreation_visits
3   1936             57775
4   1937             75434
5   1938            120692
6   1939            116516
7   1940            117751
..   ...               ...
85  2018            533206
86  2019            551590
87  2020            290392
88  2021            515774
89  2022            663147

[87 rows x 2 columns]


In [8]:
mammoth_df_1994.describe()

,year,recreation_visits
count,87,87
unique,87,87
top,1936,57775
freq,1,1


In [9]:
mammoth_df_1994.dtypes

year                 object
recreation_visits    object
dtype: object

Both columns are objects and I will need them to be numeric values to do the analyses I want to run. Time to convert!

In [10]:
mammoth_df_1994.replace(',','', regex=True, inplace=True)
mammoth_df_1994.head()

,year,recreation_visits
3,1936,57775
4,1937,75434
5,1938,120692
6,1939,116516
7,1940,117751


In [11]:
mammoth_df_1994.dtypes

year                 int64
recreation_visits    int64
dtype: object

Now I need to filter all the visitor data so I can compare it to the gas prices. Gas prices don't start until 1994, so I will drop everything prior in the visitor set. 

In [12]:
mammoth_filter = mammoth_df_1994[mammoth_df_1994.year >= 1994]
mammoth_filter.head()


,year,recreation_visits
61,1994,2009935
62,1995,1935709
63,1996,1896829
64,1997,1997658
65,1998,2113992


Back to the gas info. Time to clean it a little. The first 3 rows aren't needed. 

In [13]:
gas_df.drop([0,1,2], axis=0, inplace=True)
gas_df.head()

,Back to Contents,Data 1: U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)
3,1994-06-30 00:00:00,1.078
4,1995-06-30 00:00:00,1.158
5,1996-06-30 00:00:00,1.245
6,1997-06-30 00:00:00,1.244
7,1998-06-30 00:00:00,1.072


The first column is coming up with the name "Back to Contents"? Okay. Let's rename that to year. And that second column name is a doozy. That will get changed too. 

In [14]:
gas_df.rename(columns={gas_df.columns[0]: 'year', gas_df.columns[1]: 'price'}, inplace=True)
print(gas_df.columns)

Index(['year', 'price'], dtype='object')


In [15]:
gas_df.head()

,year,price
3,1994-06-30 00:00:00,1.078
4,1995-06-30 00:00:00,1.158
5,1996-06-30 00:00:00,1.245
6,1997-06-30 00:00:00,1.244
7,1998-06-30 00:00:00,1.072


The year column needs to be drilled down to just the year so I can merge the two dataframes together. 

In [16]:
gas_df.dtypes

year     object
price    object
dtype: object

In [17]:
gas_df['year']= pd.to_datetime(gas_df['year'])
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 3 to 31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    29 non-null     datetime64[ns]
 1   price   29 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 596.0+ bytes


In [18]:
gas_df.head()

,year,price
3,1994-06-30,1.078
4,1995-06-30,1.158
5,1996-06-30,1.245
6,1997-06-30,1.244
7,1998-06-30,1.072


I need to pull just the year out of the year column so I can merge the two dataframes together. 

In [19]:
gas_df['year'] = gas_df['year'].dt.strftime('%Y')
print(gas_df)

    year  price
3   1994  1.078
4   1995  1.158
5   1996  1.245
6   1997  1.244
7   1998  1.072
8   1999  1.176
9   2000  1.523
10  2001   1.46
11  2002  1.386
12  2003  1.603
13  2004  1.895
14  2005  2.314
15  2006  2.618
16  2007  2.843
17  2008  3.299
18  2009  2.406
19  2010  2.835
20  2011  3.576
21  2012   3.68
22  2013  3.575
23  2014  3.437
24  2015   2.52
25  2016   2.25
26  2017  2.528
27  2018  2.813
28  2019  2.691
29  2020  2.258
30  2021    3.1
31  2022  4.059


In [20]:
gas_df_convert = gas_df.astype({'year':'int64','price':'object'})
gas_df_convert.head()

,year,price
3,1994,1.078
4,1995,1.158
5,1996,1.245
6,1997,1.244
7,1998,1.072


Now that the data is clean and uniform...

Feature 2: Pandas merge. 

In [21]:
combo_df = pd.merge(gas_df_convert, mammoth_filter, how = 'left', on ='year')
combo_df.head()

,year,price,recreation_visits
0,1994,1.078,2009935
1,1995,1.158,1935709
2,1996,1.245,1896829
3,1997,1.244,1997658
4,1998,1.072,2113992


I'm going to save this dataset to xlsx so I can import into Tableau to create a dashboard. 

In [22]:
combo_df.to_excel('combo_df.xlsx')

Feature 3: Plotting.  I created a tableau dashboard, found at https://public.tableau.com/app/profile/tanya.fowler/viz/VisitorstoMammothCavebyGasPrices/VisitvsPriceLineChart?publish=yes



And Feature 4: the markdown cells should count as the analysis portion of this Analysis project. And thus concludes the lesson?